In [1]:
!pip install -q -U kaggle_environments

In [1]:
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate
from pprint import pprint

In [2]:
# Функция для форматирования вывода результата
def format_res(lst, n=0):
    if lst[n] == max(lst) and lst[n]!=lst[n-1]: return '\033[32m\033[1m{}\033[0m'.format(str(lst[n]).rjust(4))
    elif lst[n] == min(lst) and lst[n]!=lst[n-1]: return '\033[31m\033[1m{}\033[0m'.format(str(lst[n]).rjust(4))
    else: return '\033[34m\033[1m{}\033[0m'.format(str(lst[n]).rjust(4))

In [3]:
%%writefile my_agents.py

import random

# Возможные варианты в игре "rps":
#0 - rock
#1 - paper
#2 - scissors

def always_rock(observation, configuration):
    return 0

def always_paper(observation, configuration):
    return 1

def always_scissors(observation, configuration):
    return 2

def not_rock(observation, configuration):
    return random.randrange(1, configuration.signs)

def not_paper(observation, configuration):
    return random.randrange(0, configuration.signs, 2)

def not_scissors(observation, configuration):
    return random.randrange(0, configuration.signs - 1)

def agent_alternation(observation, configuration):
    return observation.step % configuration.signs

def agent_random(observation, configuration):
    return random.randrange(0, configuration.signs)

def copy_opponent(observation, configuration):
    if observation.step > 0: return observation.lastOpponentAction
    return random.randrange(0, configuration.signs)

def copy_opponent_opposite(observation, configuration):
    if observation.step > 0: return (observation.lastOpponentAction + 1) % configuration.signs
    return random.randrange(0, configuration.signs)

last_change_lose_action = None
def change_if_lose(observation, configuration):
    global last_change_lose_action
    if observation.step == 0:
        last_change_lose_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_change_lose_action) in [-2, 0, 1]:
        last_change_lose_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_change_lose_action

last_change_win_action = None
def change_if_win(observation, configuration):
    global last_change_win_action
    if observation.step == 0:
        last_change_win_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_change_win_action) in [-1, 2]:
        last_change_win_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_change_win_action


agents = {
    "always_rock": always_rock,
    "always_paper": always_paper,
    "always_scissors": always_scissors,
    "not_rock": not_rock,
    "not_paper": not_paper,
    "not_scissors": not_scissors,
    "agent_alternation": agent_alternation,
    "agent_random": agent_random,
    "copy_opponent": copy_opponent,
    "copy_opponent_opposite": copy_opponent_opposite,
    "change_if_lose": change_if_lose,
    "change_if_win": change_if_win
}

Writing my_agents.py


In [4]:
from my_agents import agents as all_agents

In [5]:
# Создание словаря агентов
competition_result = {}
for each_agent in all_agents:
    competition_result[each_agent] = 0

# Верхняя строка таблицы результатов
print(f"\033[1m{'Пара агентов'.ljust(25)}: {'Счёт'.ljust(4)}\033[0m")
print()

# Соревнования каждый с каждым number_competitions раз
# (для более точных результатов)
number_competitions = 3
for _ in range(number_competitions):
    iteration = -1
    
    for agent_1 in all_agents.values():
        iteration += 1
        
        for agent_2 in dict(list(all_agents.items())[iteration:]).values():
            if agent_1 != agent_2:

                result = list(map(int, evaluate(
                    "rps",
                    [agent_1, agent_2],
                    configuration={"episodeSteps": 100}
                )[0]))

                agent_1_name = agent_1.__name__
                agent_2_name = agent_2.__name__

                competition_result[agent_1_name] += result[0]
                competition_result[agent_2_name] += result[1]

                print(f'{agent_1_name.ljust(25)}: {format_res(result, n=0)}')
                print(f'{agent_2_name.ljust(25)}: {format_res(result, n=1)} \n')

Пара агентов             : Счёт

always_rock              :  -99
always_paper             :   99 

always_rock              :   99
always_scissors          :  -99 

always_rock              :    0
not_rock                 :    0 

always_rock              :   53
not_paper                :  -53 

always_rock              :  -53
not_scissors             :   53 

always_rock              :    0
agent_alternation        :    0 

always_rock              :    0
agent_random             :    0 

always_rock              :    0
copy_opponent            :    0 

always_rock              :  -97
copy_opponent_opposite   :   97 

always_rock              :  -98
change_if_lose           :   98 

always_rock              :   99
change_if_win            :  -99 

always_paper             :  -99
always_scissors          :   99 

always_paper             :  -34
not_rock                 :   34 

always_paper             :    0
not_paper                :    0 

always_paper             :   45
not_scissor

always_scissors          :    0
copy_opponent            :    0 

always_scissors          :  -99
copy_opponent_opposite   :   99 

always_scissors          :  -98
change_if_lose           :   98 

always_scissors          :    0
change_if_win            :    0 

not_rock                 :  -22
not_paper                :   22 

not_rock                 :   26
not_scissors             :  -26 

not_rock                 :    0
agent_alternation        :    0 

not_rock                 :    0
agent_random             :    0 

not_rock                 :    0
copy_opponent            :    0 

not_rock                 :  -26
copy_opponent_opposite   :   26 

not_rock                 :  -30
change_if_lose           :   30 

not_rock                 :    0
change_if_win            :    0 

not_paper                :  -36
not_scissors             :   36 

not_paper                :    0
agent_alternation        :    0 

not_paper                :    0
agent_random             :    0 

not_paper 

agent_alternation        :   97
copy_opponent            :  -97 

agent_alternation        :    0
copy_opponent_opposite   :    0 

agent_alternation        :    0
change_if_lose           :    0 

agent_alternation        :    0
change_if_win            :    0 

agent_random             :    0
copy_opponent            :    0 

agent_random             :    0
copy_opponent_opposite   :    0 

agent_random             :    0
change_if_lose           :    0 

agent_random             :    0
change_if_win            :    0 

copy_opponent            :  -49
copy_opponent_opposite   :   49 

copy_opponent            :  -50
change_if_lose           :   50 

copy_opponent            :    0
change_if_win            :    0 

copy_opponent_opposite   :    0
change_if_lose           :    0 

copy_opponent_opposite   :   98
change_if_win            :  -98 

change_if_lose           :   98
change_if_win            :  -98 



In [6]:
sorted_competition_result = {}
sorted_agents = sorted(competition_result, key=competition_result.get, reverse=True)

for agent in sorted_agents:
    sorted_competition_result[agent] = competition_result[agent]

# pprint(sorted_competition_result, sort_dicts=False)

column_numbers = [i for i in range(1, len(sorted_competition_result)+1)]
number = iter(column_numbers)

print('\033[1m ТУРНИРНАЯ ТАБЛИЦА \033[0m'.rjust(35))
for agent, result in sorted_competition_result.items():
    print(f'{str(next(number)).rjust(3)}. {agent.ljust(22)} - {str(result).rjust(4)}')

         ТУРНИРНАЯ ТАБЛИЦА 
  1. change_if_lose         - 1561
  2. copy_opponent_opposite - 1460
  3. agent_alternation      -  293
  4. agent_random           -    0
  5. not_paper              -  -26
  6. not_scissors           - -161
  7. not_rock               - -214
  8. always_scissors        - -398
  9. always_paper           - -485
 10. copy_opponent          - -591
 11. always_rock            - -604
 12. change_if_win          - -835
